<a href="https://colab.research.google.com/github/pararthdave/ResumeParser/blob/maulik/Language_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pdfminer.six
!pip3 install phonenumbers

In [ ]:
path="/content/drive/MyDrive/ResumeParser/Dataset/dataset.zip"
!cp -rv $path .

'/content/drive/MyDrive/ResumeParser/Dataset/dataset.zip' -> './dataset.zip'


In [ ]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: dataset/
  inflating: dataset/Harika Sonnathi Resume 2022 2.pdf  
  inflating: dataset/KetanPawarResume.pdf  
  inflating: dataset/Mandar bhoir Resume (1)_compressed.pdf  
  inflating: dataset/pararthdave_Resume.pdf  
  inflating: dataset/Priyanka_RaikwarRESUME.pdf  
  inflating: dataset/priyanka_resume(8).pdf  
  inflating: dataset/ravisatvik.192_Resume.pdf  
  inflating: dataset/YOGAPRIYA H_RESUME.pdf  


In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
import sys,os,io
def split(delimiters, string, maxsplit=0):
    import re
    regexPattern = '|'.join(map(re.escape, delimiters))
    return re.split(regexPattern, string, maxsplit)



def get_lang_list():
    filePath = './Languages.csv'
    df = pd.read_csv(filePath)
    # str.replace('; ', ', ') and then a str.split(', ')
    df['Language name '].replace(to_replace ="marathi (marāṭhī)",
                 value ="marathi")
    # languages = df.iloc[:, [3]].to_list()
    lang_list =df['Language name '].to_list()
    all_lang_list=[]
    for i in lang_list:
        i=i.strip().lower()
        i = split(' ,;()', i)
        if(type(i) == list):
            for x in i:
                if(x!=''):
                    all_lang_list.append(x)
        else:
            all_lang_list.append(i)
    
    return all_lang_list

def ProperNounExtractor(text):
    lst1=[]
    string=" ".join(map(str, text))
    string=string.replace('[','')
    string=string.replace(']','')
    string=string.replace("'",'')
    sentences = nltk.sent_tokenize(string)
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        tagged = nltk.pos_tag(words)
        for (word, tag) in tagged:
            if tag == 'NN': # If the word is a proper noun
                print(word)
lst=get_lang_list()
tokenized_sents = [word_tokenize(i) for i in lst]
old_stdout = sys.stdout
new_stdout = io.StringIO()
sys.stdout = new_stdout
ProperNounExtractor(tokenized_sents)

output = new_stdout.getvalue()

sys.stdout = old_stdout

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
output.replace('\n',' ')

'aymara azerbaijani bashkir bihari bislama bambara bengali bangla standard breton bosnian catalan chechen chamorro cree czech church church chuvash welsh divehi dhivehi dzongkha ewe esperanto basque farsi fula fulah pulaar guaraní gujarati manx hausa hindi hiri motu creole herero interlingua interlingue igbo inupiaq ido inuktitut kongo kikuyu gikuyu kwanyama kuanyama kazakh kalaallisut khmer komi kyrgyz letzeburgesch ganda lao malagasy māori malayalam marathi marāṭhī malay nauru bokmål nepali dutch nynorsk south navaho chichewa chewa ojibwe ojibwa oromo panjabi punjabi pāli pashto pushto quechua romansh kirundi kinyarwanda sanskrit saṁskṛta sindhi sami sango slovene samoan shona somali swati sotho swahili tamil telugu tajik thai tonga tonga tsonga tatar twi uyghur urdu venda volapük walloon wolof zulu '

In [ ]:
import re
import subprocess
import os
import pdfminer
from pdfminer.high_level import extract_text
import phonenumbers
import nltk
import pandas as pd
import spacy
import pprint
from spacy.matcher import Matcher


nlp = spacy.load('en_core_web_sm')

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

RESERVED_WORDS = [
    'achievement',
    'accomplishment',
]

def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

def doc_to_text_catdoc(file_path):
    try:
        process = subprocess.Popen(  # noqa: S607,S603
            ['catdoc', '-w', file_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
        )
    except (
        FileNotFoundError,
        ValueError,
        subprocess.TimeoutExpired,
        subprocess.SubprocessError,
    ) as err:
        return (None, str(err))
    else:
        stdout, stderr = process.communicate()

    return (stdout.strip(), stderr.strip())

def matchLang(lang):
    languages = lang

    pattern = [{'POS': 'PROPN'}]

    matcher = Matcher(nlp.vocab)
    matcher.add('langMatch', [pattern])
    
    for lang in languages:
        matches.matcher(nlp(lang))
    matches = matcher(nlp(languages))
    
    # print(matches)
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        print(span.text)
        return span.text

def extract_language(input_text):
    languages = get_lang_list()
    # print(languages)
    nlp_text = nlp(input_text)
    noun_chunks = nlp_text.noun_chunks
    # print(type(noun_chunks))
    tokens = [token.text for token in nlp_text if not token.is_stop]
    master_language = []


    for token in tokens:
        if token.lower() in languages:
            master_language.append(token)
    
    for token in noun_chunks:
        token = token.text.lower().strip()
        # print(token)
        for lang in languages:
            if(token==lang):
                master_language.append(token)
            # if(lang.find(token) != -1):
            #     master_language.append(token)

        # if token in languages:
        #     master_language.append(token)
    return [i.capitalize() for i in set([i.lower() for i in master_language])]

        

if __name__ == '__main__':
    directory = '/content/dataset/'

    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        text = extract_text_from_pdf(f)
        # print(matchLang())
        print(extract_language(text))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[]
[]
['Standard']
[]
[]
['Telugu', 'Central']
['Japanese', 'Hindi', 'Marathi', 'English']
['Modern']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'996 668'.split()[0]

'996'